In [5]:
def get_assignments():

    from IPython.display import display, HTML, clear_output
    import ipywidgets as widgets
    from datetime import datetime
    from dateutil import parser, tz
    import pandas as pd
    import current_course as cc
    import import_ipynb
    import GradingTools as gt


    #setup
    course = cc.read_current_course()
    if course is None:
        raise Exception("You must select a current course first")

    templates = gt.get_templates()
    assignments = gt.client.get_course_assignments(course)
    roster = gt.client.get_course_roster(course)
    units = assignments['unit_name'].unique().tolist()
    assignment_names = assignments['assignment_name'].unique().tolist()
    gt.initialize_directories()

    # widgets
    current_course_text = widgets.Text(
        value=course,
        description='Course:',
        disabled=True
    )

    unit_dropdown = widgets.Dropdown(
        options=units,
        value=units[0],
        description='Lesson:',
        layout={'width': 'max-content'},
        disabled=False
    )

    assignment_name_dropdown = widgets.Dropdown(
        options=assignment_names,
        value=assignment_names[0],
        description='Assignment:',
        layout={'width': 'max-content'},
        disabled=False
    )

    assignment_date_text = widgets.Text(
        value=assignments[ assignments['assignment_name'] == assignment_name_dropdown.value]['due_date'].values[0],
        description='Due Date:',
        disabled=True
    )

    assignment_points_text = widgets.Text(
        value=str(assignments[ assignments['assignment_name'] == assignment_name_dropdown.value]['points'].values[0]),
        description='Total Points:',
        layout={'width': 'max-content'},
        disabled=True
    )

    assignment_lms_column_text = widgets.Text(
        value=str(assignments[ assignments['assignment_name'] == assignment_name_dropdown.value]['lms_column_id'].values[0]),
        description='BB Column:',
        layout={'width': 'max-content'},
        disabled=True
    )

    template_dropdown = widgets.Dropdown(
        options=templates,
        value=templates[0],
        description='Template:',
        layout={'width': 'max-content'},
        disabled=False
    )


    display_submissions_button = widgets.Button(
        description="Display",        
        icon="table",
        button_style='info',
        layout={'width': 'max-content'},
        disabled = False)

    fetch_submissions_button = widgets.Button(
        description="Fetch",        
        icon="download",
        button_style='primary',
        layout={'width': 'max-content'},
        disabled = False)

    open_submissions_folder_button = widgets.Button(
        description="Open Folder",
        icon="folder",
        button_style='success',
        layout={'width': 'max-content'},
        disabled = False)

    clear_submissions_button = widgets.Button(
        description="Clear Folder",
        icon="trash",
        button_style='danger',
        layout={'width': 'max-content'},
        disabled = False)


    output = widgets.Output()

    # functions
    def unit_dropdown_on_change(*args):
        assignment_name_dropdown.options = assignments[assignments['unit_name'] == unit_dropdown.value].sort_values('assignment_name')['assignment_name'].unique().tolist()

    def assignment_name_dropdown_on_change(*args):
        assignment_date_text.value = assignments[ assignments['assignment_name'] == assignment_name_dropdown.value]['due_date'].values[0]
        assignment_points_text.value=str(assignments[ assignments['assignment_name'] == assignment_name_dropdown.value]['points'].values[0])
        assignment_lms_column_text.value=str(assignments[ assignments['assignment_name'] == assignment_name_dropdown.value]['lms_column_id'].values[0])
        display_submissions_button_on_click()
        #calc_status(unit_dropdown.value,assignment_dropdown.value,assignment_bbcolumn.value)

    def fetch_submissions_button_on_click(*args):
        with output:
            clear_output()
            gt.fetch_assignments(course, assignment_name_dropdown.value, template_dropdown.value)
            display_df = gt.display_submissions(course, assignment_name_dropdown.value)
            display(display_df[['view_submission', 'late','submitted_on','last_fetch','template_file']].style)

    def open_submissions_folder_button_on_click(*args):
        with output:
            clear_output()
            url = gt.get_assignments_folder_url(course, assignment_name_dropdown.value)
            display(HTML( '<b>Click to Open</b>: <a href="{}" rel="noopener noreferrer" target="_blank">{}</a>'.format(url,url) ) )

    def clear_submissions_button_on_click(*args):
        with output:
            clear_output()
            path = gt.clear_assignments_folder(course, assignment_name_dropdown.value)
            display(f"Folder {path} cleared.")

    def display_submissions_button_on_click(*args):
        with output:
            clear_output()
            if assignment_name_dropdown.value.upper().startswith("LAB"):
                display_df = gt.display_lab_submissions(course, assignment_name_dropdown.value)
            elif assignment_name_dropdown.value.upper().startswith("HW"):
                #TODO  Display Homework Submissions
                display_df = gt.display_hw_submissions(course, assignment_name_dropdown.value)
            else:
                display_df = gt.display_submissions(course, assignment_name_dropdown.value)

            if len(display_df) == 0:
                display("Nothing do Display. Fetch first!")
            else:
                if assignment_name_dropdown.value.upper().startswith("LAB"):
                    display(display_df[['view_submission', 'late','submitted_on','last_fetch','template_file', 'lab_check', 'comfort', 'questions']].style.set_properties(**{'text-align': 'left'}))
                elif assignment_name_dropdown.value.upper().startswith("HW"):
                    #TODO: Homework Submissions Columns
                    display(display_df[['view_submission', 'late','submitted_on','last_fetch','template_file', 'problem_analysis', 'homework_check', 'metacognition', 'comfort']].style.set_properties(**{'text-align': 'left'}))
                else:
                    display(display_df[['view_submission', 'late','submitted_on','last_fetch','template_file']].style.set_properties(**{'text-align': 'left'}))
    # handlers
    unit_dropdown.observe(unit_dropdown_on_change)
    assignment_name_dropdown.observe(assignment_name_dropdown_on_change)
    fetch_submissions_button.on_click(fetch_submissions_button_on_click)
    clear_submissions_button.on_click(clear_submissions_button_on_click)
    open_submissions_folder_button.on_click(open_submissions_folder_button_on_click)
    display_submissions_button.on_click (display_submissions_button_on_click)

    # layouts
    display(HTML(f"<h3>Get Course Assignments</h3>"))
    display(current_course_text)
    display(widgets.HBox([unit_dropdown,assignment_name_dropdown]))
    display(assignment_date_text)
    display(widgets.HBox([assignment_points_text, assignment_lms_column_text]))
    display(template_dropdown)
    display(widgets.HBox([display_submissions_button,fetch_submissions_button,open_submissions_folder_button,clear_submissions_button]))
    display(output)

    #initialize
    unit_dropdown_on_change()
    assignment_name_dropdown_on_change()
    display_submissions_button_on_click()


In [6]:
if __name__ == '__main__':
    get_assignments()

Text(value='tst101-spring2024', description='Course:', disabled=True)

Text(value='2024-01-05 23:59', description='Due Date:', disabled=True)

Dropdown(description='Template:', layout=Layout(width='max-content'), options=('./templates/Homework.ipynb', '…

Output()